In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [2]:
# Define dataset paths
train_dir = '../ARchitect_Dataset/Train'
val_dir = '../ARchitect_Dataset/Validate'
test_dir = '../ARchitect_Dataset/Test'



In [3]:
# Define hyperparameters
img_size = 150
batch_size = 32
epochs = 100
dropout_rate = 0.5



In [4]:
# Define data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')


Found 1000 images belonging to 2 classes.
Found 210 images belonging to 2 classes.
Found 210 images belonging to 2 classes.


In [5]:

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout_rate),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout_rate),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(dropout_rate),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(dropout_rate),
    Dense(1, activation='sigmoid')
])


In [6]:

# Compile the model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [7]:
# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator)



Epoch 1/100


2023-03-15 11:25:30.776017: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


32/32 [==============================] - 14s 408ms/step - loss: 2.2894 - accuracy: 0.5070 - val_loss: 0.6928 - val_accuracy: 0.6095
Epoch 2/100
32/32 [==============================] - 14s 449ms/step - loss: 0.6936 - accuracy: 0.5290 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 3/100
32/32 [==============================] - 13s 391ms/step - loss: 0.7021 - accuracy: 0.5640 - val_loss: 0.6889 - val_accuracy: 0.6619
Epoch 4/100
32/32 [==============================] - 13s 410ms/step - loss: 0.7160 - accuracy: 0.5700 - val_loss: 0.6592 - val_accuracy: 0.5095
Epoch 5/100
32/32 [==============================] - 13s 394ms/step - loss: 0.7447 - accuracy: 0.5790 - val_loss: 0.6690 - val_accuracy: 0.6333
Epoch 6/100
32/32 [==============================] - 12s 380ms/step - loss: 0.6630 - accuracy: 0.6270 - val_loss: 0.6517 - val_accuracy: 0.5857
Epoch 7/100
32/32 [==============================] - 13s 405ms/step - loss: 0.6620 - accuracy: 0.6300 - val_loss: 0.6385 - val_accuracy: 0.7095
Epoc

In [8]:
# Evaluate the model on test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)



7/7 [==============================] - 1s 143ms/step - loss: 0.4806 - accuracy: 0.8286
Test accuracy: 0.8285714387893677


In [9]:
# Save the model
model.save('furniture_classification_model.h5')

In [7]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

test_dir = '/Users/sarithwijesundera/Downloads/test_imgs'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary')

model = tf.keras.models.load_model('furniture_classification_model.h5')

predictions = model.predict(test_generator)

for i, prediction in enumerate(predictions):
    if prediction > 0.5:
        print("Image", i+1, "is organized with a probability of", prediction[0])
    else:
        print("Image", i+1, "is messy with a probability of", 1-prediction[0])


Found 2 images belonging to 2 classes.
2/2 [==============================] - 0s 12ms/step
Image 1 is messy with a probability of 0.8700495064258575
Image 2 is organized with a probability of 0.6544863


In [38]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the trained model
model = tf.keras.models.load_model('furniture_classification_model.h5')

# Define the image path to test
img_path = '/Users/sarithwijesundera/Downloads/DSC_67051.jpg'

# Preprocess the image
img = Image.open(img_path).resize((150,150))
img = np.array(img)/255.0
img = img.reshape(1,150,150,3)

# Use the model to predict the category of the image
pred = model.predict(img)

# Print the prediction result with the percentage
if pred[0][0] < 0.5:
    print("The image is messy with {:.2%} confidence.".format(1-pred[0][0]))
else:
    print("The image is organized with {:.2%} confidence.".format(pred[0][0]))


1/1 [==============================] - 0s 57ms/step
The image is messy with 66.17% confidence.
